## Messari standardised subgraphs

https://github.com/messari/subgraphs

### Ubeswap Celo

12/09/22

In [3]:
import pandas as pd
import numpy as np
import time
import requests
import datetime as dt

In [4]:
endpoint = 'https://api.thegraph.com/subgraphs/name/messari/ubeswap-celo'

In [5]:
#Check if subgraph is up to date
query = """{
  _meta{
    block{
      timestamp
  		hash
      number
    }
    hasIndexingErrors
    deployment
  }
}"""
request = requests.post(endpoint, json={'query':query})
data = request.json()
#Check if it is ok to run i.e. within 5 days of today
print(dt.datetime.fromtimestamp(int(data['data']['_meta']['block']['timestamp'], base = 0)))
int(data['data']['_meta']['block']['timestamp'], base = 0) > (dt.datetime.today().timestamp() - 3600*24*5)

2022-09-12 17:25:49


True

6-Month Average Daily Volume

In [6]:
query = """{
  financialsDailySnapshots(first: 183, orderBy: timestamp, orderDirection: desc){
    id
    protocol{
      name
    }
    dailyVolumeUSD
    dailyTotalRevenueUSD
    dailySupplySideRevenueUSD
    dailyProtocolSideRevenueUSD
    timestamp
    blockNumber
  }
}
"""

In [7]:
request = requests.post(endpoint, json={'query':query})
if request.ok:
    data = request.json()
else:
    print('Retry!')

In [8]:
df = pd.json_normalize(data['data']['financialsDailySnapshots'])

In [9]:
df['dailyVolumeUSD'].astype(float).mean()

892077.1628380137

6-Month Average TVL

In [10]:
#DefiLlama https://defillama.com/protocols/dexes 
lma = pd.read_csv("https://api.llama.fi/dataset/ubeswap.csv")

In [11]:
lma

,Unnamed: 0,Date,Timestamp,Ubeswap,Ubeswap.1,Ubeswap.2,Ubeswap.3,Ubeswap.4,Ubeswap.5,Ubeswap.6,...,Ubeswap.84,Ubeswap.85,Ubeswap.86,Ubeswap.87,Ubeswap.88,Ubeswap.89,Ubeswap.90,Ubeswap.91,Ubeswap.92,Ubeswap.93
0,NaN,NaN,NaN,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,...,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes,Dexes
1,NaN,NaN,NaN,celo,Total,celo,celo,celo,celo,celo,...,Total,Total,Total,Total,Total,Total,Total,Total,Total,Total
2,NaN,NaN,NaN,TVL,TVL,Tokens(USD),Tokens(USD),Tokens(USD),Tokens(USD),Tokens(USD),...,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens,Tokens
3,NaN,NaN,NaN,NaN,NaN,USDT,celo,CEUR,mceur,tether,...,ethereum,POOF,PREMIO,moola-celo-dollars,CELO,usd-coin,CUSD,mcelo,UBE,CREAL
4,NaN,22/02/2021,1.613952e+09,0.01001024525322803,0.01001024525322803,0.01001024525322803,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562,NaN,08/09/2022,1.662595e+09,5464587.848124873,5464587.848124873,NaN,NaN,483.306069543005,1968257.8516175062,15.029263224485463,...,121.2997290301077,9267927.345678993,9440717.502169391,732554.8567348571,2613048.782920584,18.140877503508676,145388.43476376968,216434.13720917652,408831.570284287,NaN
563,NaN,09/09/2022,1.662682e+09,5607661.953285333,5607661.953285333,NaN,NaN,507.40111589390256,2002509.0058166739,15.044292487709948,...,120.92375513861553,9277519.165774042,8662342.53793398,733919.4387414741,2606203.630844389,18.140877503508676,147180.96119504058,215092.48003845187,423750.62642670196,16.404673607590063
564,NaN,10/09/2022,1.662768e+09,5773613.052006499,5773613.052006499,NaN,NaN,513.0804266775413,2049076.6027275568,15.044292487709948,...,118.02105604254925,9292591.494356267,8663085.793869982,735407.529570055,2569120.8021960394,18.140877503508676,150015.52102189665,212011.36410471474,422738.98945727444,16.19257525285381
565,NaN,11/09/2022,1.662854e+09,5750173.050259095,5750173.050259095,NaN,NaN,514.334663470112,2036385.2328312441,15.044292487709948,...,115.94262811044989,9292316.648520757,8663084.556851309,735763.710692097,2568641.73378544,18.140877503508676,150802.8316736963,212298.00214157347,422706.13126003166,16.296267763778708


In [12]:
lma.iloc[:,4]

0                    Dexes
1                    Total
2                      TVL
3                      NaN
4      0.01001024525322803
              ...         
562      5464587.848124873
563      5607661.953285333
564      5773613.052006499
565      5750173.050259095
566      5675032.586929696
Name: Ubeswap.1, Length: 567, dtype: object

In [13]:
#Just eth currently can take total by changing the header
lma.iloc[-183:, 4].astype(float).mean()

13252090.531421017

Largest Pool Concentration

In [14]:
df = pd.DataFrame()
ts = 0

while True:
    sl = len(df)
    query = """
{
  liquidityPools(first:1000, orderBy: createdTimestamp, orderDirection: asc, where: {createdTimestamp_gt: """+f'"{ts}"'+"""}){
    id
    name
    totalValueLockedUSD
    cumulativeVolumeUSD
    createdTimestamp
    createdBlockNumber
  }
}
"""
    request = requests.post(endpoint, json={'query':query})
    data = request.json()

    pool = pd.json_normalize(data['data']['liquidityPools'])
    df = df.append(pool)
    df.drop_duplicates(inplace=True)
    if len(df) == sl:
      break
    ts = df['createdTimestamp'].max()


/tmp/ipykernel_75326/2291472997.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pool)


In [15]:
df['totalValueLockedUSD'].astype(float).max()/df['totalValueLockedUSD'].astype(float).sum()

0.23256924827913233

In [16]:
df['totalValueLockedUSD'].astype(float).nlargest(10).sum()/df['totalValueLockedUSD'].astype(float).sum()

0.7883377120486317

Liquidity Pools with > $1mn TVL

In [17]:
len(df.loc[df['totalValueLockedUSD'].astype(float) > 1000000])

2

Monthly Active Users

In [18]:
query = """{
  usageMetricsDailySnapshots(first: 183, orderBy: timestamp, orderDirection: desc){
    timestamp
    blockNumber
    id
    dailyActiveUsers
    dailySwapCount
    dailyDepositCount
    dailyWithdrawCount 
  }
}
"""

In [19]:
request = requests.post(endpoint, json={'query':query})
data = request.json()

In [20]:
df = pd.json_normalize(data['data']['usageMetricsDailySnapshots'])

In [21]:
df['dailyActiveUsers'].astype(float).mean()

534.8142076502733

Implied slippage

In [22]:
query = """
{
  liquidityPools(first:10, orderBy: totalValueLockedUSD, orderDirection: desc){
    id
    name
    totalValueLockedUSD
    cumulativeVolumeUSD
    createdTimestamp
    createdBlockNumber
    inputTokens{
      id
      name
      symbol
      lastPriceUSD
      decimals
    }
    inputTokenBalances
  }
}
"""
request = requests.post(endpoint, json={'query':query})
if request.ok:
    data = request.json()

In [23]:
data

{'data': {'liquidityPools': [{'id': '0xf94fea0c87d2b357dc72b743b45a8cb682b0716e',
    'name': 'Ubeswap Moola interest bearing CUSD/Moola interest bearing CEUR',
    'totalValueLockedUSD': '1324445.138787230835229968',
    'cumulativeVolumeUSD': '50762177.63214418639434330591503276',
    'createdTimestamp': '1634104380',
    'createdBlockNumber': '9305489',
    'inputTokens': [{'id': '0x918146359264c492bd6934071c6bd31c854edbc3',
      'name': 'Moola interest bearing CUSD',
      'symbol': 'mCUSD',
      'lastPriceUSD': '1',
      'decimals': 18},
     {'id': '0xe273ad7ee11dcfaa87383ad5977ee1504ac07568',
      'name': 'Moola interest bearing CEUR',
      'symbol': 'mCEUR',
      'lastPriceUSD': '1.00824758111311471509639811567899',
      'decimals': 18}],
    'inputTokenBalances': ['662222569393615417614984',
     '656805512652473251188049']},
   {'id': '0xb460f9ae1fea4f77107146c1960bb1c978118816',
    'name': 'Ubeswap Celo Gold/Moola interest bearing CUSD',
    'totalValueLockedUSD': '1

In [24]:
slip1000 = []
slip10000 = []
slip100000 = []

for pool in data['data']['liquidityPools']:
    token0 = pool['inputTokens'][0]
    token0_reserveUSD = (float(pool['inputTokenBalances'][0])/10**token0['decimals'])*float(token0['lastPriceUSD'])
    
    token1 = pool['inputTokens'][1]
    token1_reserveUSD = (float(pool['inputTokenBalances'][1])/10**token1['decimals'])*float(token1['lastPriceUSD'])

    try:
        pi1000 = 1000/token0_reserveUSD
    except:
        pi1000 = 1000/token1_reserveUSD
    
    try:
        pi10000 = 10000/token0_reserveUSD
    except:
        pi10000 = 10000/token1_reserveUSD
    
    try:
        pi100000 = 100000/token0_reserveUSD
    except:
        pi100000 = 100000/token1_reserveUSD
    
    slip1000.append(pi1000)
    slip10000.append(pi10000)
    slip100000.append(pi100000)


In [25]:
np.mean(slip1000)

0.007177492550630558

In [26]:
np.mean(slip10000)

0.07177492550630557

In [27]:
np.mean(slip100000)

0.7177492550630558